# JSON parser
- Allows users to specify an arbitrary JSON schema and query LLMs for output that conform to that schema

In [10]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini",temperature=0)

In [5]:
# Define a structure
class Joke(BaseModel):
    setup: str = Field(..., description="The setup of the joke")
    punchline: str = Field(..., description="The punchline of the joke")

In [6]:
joke_query = "Tell me a joke"

#setup up a parser + inject instructions into the prompt template
parser = JsonOutputParser(pydantic_object=Joke)
prompt = PromptTemplate(
    template="Answer the question based on the context below.\n\n{format_instructions}\n\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser
chain.invoke({"question": joke_query})


{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

In [23]:
def call_json_output_parser():
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system","You are a helpful assistant to extract information from the following phrase\nFormatting Instructions:{formatting_instructions}",
            ),
            ("user", "{phrase}"),
        ]
    )
    class Person(BaseModel):
        name: str = Field(description="The name of the person")
        age: int = Field(description="The age of the person")
       
    parser = JsonOutputParser(pydantic_object=Person)
    chain = prompt | llm | parser
    resp = chain.invoke({"phrase": "John is 30 years old", "formatting_instructions": parser.get_format_instructions()})
    return resp

In [24]:
response = call_json_output_parser()
print(response)

{'name': 'John', 'age': 30}


In [26]:
from typing import List
prompt = prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system","You are a helpful assistant to extract information from the following phrase\nFormatting Instructions:{formatting_instructions}",
            ),
            ("user", "{phrase}"),
        ]
    )
class FoodReceipes(BaseModel):
    food:str = Field(description="Name of the food")
    ingredients:List[str] = Field(description="List of ingredients")

receipe_parser = JsonOutputParser(pydantic_object=FoodReceipes)
chain = prompt | llm | receipe_parser
response = chain.invoke({"phrase": "Prepare the Margherita pizza with tomatoes, onions, cheese, bell-pepper",
                        "formatting_instructions":receipe_parser.get_format_instructions()})
print(response)

{'food': 'Margherita pizza', 'ingredients': ['tomatoes', 'onions', 'cheese', 'bell-pepper']}


# Comma Separated List output Parser

In [12]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [17]:
def call_list_output_parser():
    prompt = ChatPromptTemplate.from_messages([
        ("system","Generate a list of 10 synonyms forr the following word. Ruturn the result in a comma separated list."),
        ("human", "{word}")
    ]     
    )

    parser = CommaSeparatedListOutputParser()
    chain = prompt | llm | parser
    res = chain.invoke({"word": "Happy"})
    return res

response = call_list_output_parser()
print(response)
print(type(response))

['Joyful', 'Cheerful', 'Elated', 'Content', 'Delighted', 'Pleased', 'Blissful', 'Gleeful', 'Ecstatic', 'Jubilant']
<class 'list'>
